# Shopify Subdomain Takeover Scanner

**Dataset-based workflow:**
1. Load CSV dataset (`/kaggle/input/all-leads-merged/results.csv`)
2. Filter: Shopify stores + exclude myshopify.com + exclude 200/429
3. Sort by Est Monthly Page Views (highest traffic first)
4. Select range to scan (e.g., rows 1-100)
5. Deep scan with full mode

**Total time:** ~7-9 hours for 10,000 domains (within Kaggle's 12-hour limit)

## Cell 1: Clone Project from GitHub

In [ ]:
%%bash
WORKDIR="/kaggle/working"
PROJECT_DIR="$WORKDIR/subdomain-playground"

echo "=========================================="
echo "Cloning Project from GitHub"
echo "=========================================="

mkdir -p "$WORKDIR"
cd "$WORKDIR"

if [ -d "$PROJECT_DIR" ]; then
    echo "Removing existing copy..."
    rm -rf "$PROJECT_DIR"
fi

git clone --depth 1 https://github.com/sayihhamza/subdomain-playground.git subdomain-playground

if [ ! -d "$PROJECT_DIR" ]; then
    echo "✗ Clone failed!"
    exit 1
fi

cd "$PROJECT_DIR"

echo ""
echo "✓ Project cloned successfully!"
echo ""
echo "Project structure:"
ls -lh | head -20

## Cell 2: Install Python Dependencies

In [ ]:
%%bash
cd /kaggle/working/subdomain-playground

echo "Installing Python requirements..."
python3 -m pip install --quiet -r requirements.txt

echo ""
echo "✓ Dependencies installed!"

## Cell 3: Install Go 1.24

In [ ]:
%%bash
cd /tmp

if command -v sudo >/dev/null 2>&1; then
    SUDO="sudo"
else
    SUDO=""
fi

echo "=========================================="
echo "Installing Go 1.24.1"
echo "=========================================="

$SUDO rm -rf /usr/local/go 2>/dev/null || true

echo "Downloading Go 1.24.1..."
for i in {1..3}; do
    wget -q https://go.dev/dl/go1.24.1.linux-amd64.tar.gz -O /tmp/go.tar.gz && break || sleep 5
done

if [ ! -f /tmp/go.tar.gz ]; then
    echo "✗ Failed to download Go"
    exit 1
fi

$SUDO tar -C /usr/local -xzf /tmp/go.tar.gz
rm -f /tmp/go.tar.gz

echo ""
echo "✓ Go 1.24.1 installed!"
/usr/local/go/bin/go version

## Cell 4: Build Security Tools

Builds 5 tools (5-8 minutes):
- **subfinder**: Passive subdomain enumeration
- **findomain**: Additional passive enumeration (optional)
- **httpx**: HTTP validation
- **dnsx**: DNS + CNAME validation
- **subzy**: Takeover detection

In [ ]:
%%bash
export PATH=/usr/local/go/bin:$PATH
cd /kaggle/working/subdomain-playground

echo "=========================================="
echo "Building Security Tools"
echo "=========================================="
echo ""

mkdir -p bin

# Build Go tools
build_tool() {
    local name=$1
    local repo=$2
    local index=$3
    local total=$4
    local max_attempts=3
    
    echo "[$index/$total] Building $name..."
    
    for attempt in $(seq 1 $max_attempts); do
        if [ $attempt -gt 1 ]; then
            echo "  Retry $attempt/$max_attempts..."
            sleep 2
        fi
        
        if GOBIN=$(pwd)/bin /usr/local/go/bin/go install -v ${repo}@latest 2>&1; then
            if [ -f "bin/$name" ]; then
                echo "  ✓ $name built successfully"
                return 0
            fi
        fi
    done
    
    echo "  ✗ Failed to build $name"
    return 1
}

# Download precompiled binary
download_binary() {
    local name=$1
    local url=$2
    local index=$3
    local total=$4
    local max_attempts=3
    
    echo "[$index/$total] Downloading $name..."
    
    for attempt in $(seq 1 $max_attempts); do
        if [ $attempt -gt 1 ]; then
            echo "  Retry $attempt/$max_attempts..."
            sleep 2
        fi
        
        if curl -sL "$url" -o /tmp/${name}.zip 2>&1; then
            if unzip -q /tmp/${name}.zip -d bin/ 2>&1; then
                if [ -f "bin/$name" ]; then
                    chmod +x "bin/$name"
                    echo "  ✓ $name downloaded successfully"
                    rm -f /tmp/${name}.zip
                    return 0
                fi
            fi
        fi
    done
    
    echo "  ⚠️  $name download failed (optional)"
    rm -f /tmp/${name}.zip
    return 1
}

# Build required tools
build_tool "subfinder" "github.com/projectdiscovery/subfinder/v2/cmd/subfinder" "1" "5"
echo ""

download_binary "findomain" "https://github.com/Findomain/Findomain/releases/latest/download/findomain-linux-i386.zip" "2" "5"
echo ""

build_tool "httpx" "github.com/projectdiscovery/httpx/cmd/httpx" "3" "5"
echo ""

build_tool "dnsx" "github.com/projectdiscovery/dnsx/cmd/dnsx" "4" "5"
echo ""

build_tool "subzy" "github.com/PentestPad/subzy" "5" "5"
echo ""

echo "=========================================="
echo "Verification"
echo "=========================================="

REQUIRED="subfinder httpx dnsx subzy"
TOOLS_OK=true

for tool in $REQUIRED; do
    if [ -f "bin/$tool" ]; then
        echo "  ✓ bin/$tool"
    else
        echo "  ✗ bin/$tool MISSING"
        TOOLS_OK=false
    fi
done

[ -f "bin/findomain" ] && echo "  ✓ bin/findomain (bonus!)" || echo "  ⚠️  bin/findomain (optional)"

if [ "$TOOLS_OK" = false ]; then
    echo ""
    echo "⚠️  Required tools failed - re-run this cell"
    exit 1
fi

echo ""
echo "✓ All required tools built!"

## Cell 5: Set Environment Variables

In [ ]:
import os

os.environ['SUBFINDER_PATH'] = '/kaggle/working/subdomain-playground/bin/subfinder'
os.environ['FINDOMAIN_PATH'] = '/kaggle/working/subdomain-playground/bin/findomain'
os.environ['DNSX_PATH'] = '/kaggle/working/subdomain-playground/bin/dnsx'
os.environ['HTTPX_PATH'] = '/kaggle/working/subdomain-playground/bin/httpx'
os.environ['SUBZY_PATH'] = '/kaggle/working/subdomain-playground/bin/subzy'

os.chdir('/kaggle/working/subdomain-playground')

print("✓ Environment configured")
print(f"✓ Working directory: {os.getcwd()}")

## Cell 6: Load and Filter Dataset

**Configuration:**
- `START_ROW`: First row to scan (default: 1)
- `END_ROW`: Last row to scan (default: 100)
- `SORT_BY`: Column to sort by (default: 'Est Monthly Page Views')

**Filters applied:**
1. Is_Shopify == 'Yes'
2. Exclude *.myshopify.com
3. Exclude HTTP 200 and 429
4. Sort by specified column (descending)
5. Select rows START_ROW to END_ROW

In [ ]:
import pandas as pd

# ========================================
# CONFIGURATION
# ========================================
START_ROW = 1      # First row to scan (1-indexed)
END_ROW = 100      # Last row to scan
SORT_BY = 'Est Monthly Page Views'  # Column to sort by

print("=" * 80)
print("LOADING AND FILTERING DATASET")
print("=" * 80)
print("")

# Load CSV
csv_path = '/kaggle/input/all-leads-merged/results.csv'
print(f"Loading: {csv_path}")
df = pd.read_csv(csv_path, low_memory=False)
print(f"✓ Loaded {len(df):,} total rows")
print("")

# Filter A: Shopify stores only
is_shopify = df['Is_Shopify'] == 'Yes'

# Filter B: Exclude raw 'myshopify.com' domains
is_custom_domain = ~df['Subdomain'].str.contains('myshopify.com', case=False, na=False)

# Filter C: Exclude HTTP 200 and 429
excluded_statuses = [200, 429, '200', '429', 200.0, 429.0]
is_interesting_status = ~df['HTTP_Status'].isin(excluded_statuses)

# Apply all filters
df_filtered = df[is_shopify & is_custom_domain & is_interesting_status].copy()

print("Filters applied:")
print(f"  Original rows: {len(df):,}")
print(f"  After Shopify filter: {is_shopify.sum():,}")
print(f"  After myshopify.com exclusion: {(is_shopify & is_custom_domain).sum():,}")
print(f"  After status exclusion: {len(df_filtered):,}")
print("")

# Status breakdown
print("Status code breakdown (after filters):")
status_counts = df_filtered['HTTP_Status'].value_counts().head(10)
for status, count in status_counts.items():
    print(f"  {status}: {count:,}")
print("")

# Sort by specified column
print(f"Sorting by: {SORT_BY} (descending)")
if SORT_BY in df_filtered.columns:
    # Convert to numeric if it's a numeric column
    df_filtered['_sort_numeric'] = pd.to_numeric(
        df_filtered[SORT_BY].astype(str).str.replace(r'[^\d.]', '', regex=True),
        errors='coerce'
    ).fillna(0)
    df_sorted = df_filtered.sort_values('_sort_numeric', ascending=False)
    print(f"✓ Sorted by {SORT_BY}")
else:
    print(f"⚠️  Column '{SORT_BY}' not found - using unsorted")
    df_sorted = df_filtered
print("")

# Select range
print(f"Selecting rows {START_ROW} to {END_ROW}")
df_selected = df_sorted.iloc[START_ROW-1:END_ROW].copy()
print(f"✓ Selected {len(df_selected)} domains")
print("")

# Store for next cells (Cell 7 and Cell 8 will use their own ranges)
# This creates the base filtered/sorted dataset
df_base_filtered = df_sorted.copy()

print("=" * 80)
print("Dataset loaded and filtered")
print(f"Total available after filtering: {len(df_base_filtered):,} rows")
print("")
print("Next: Configure range in Cell 7 (preview) and Cell 8 (scan)")
print("=" * 80)

## Cell 7: Preview Full Dataset (Manual Review)

**This cell displays the complete filtered and sorted dataset before scanning.**

You can manually check subdomains while the scan runs in Cell 8.

**Columns shown:**
- Row (1-based index for easy reference)
- Subdomain (verified to be subdomain, not root domain)
- HTTP Status
- Est Monthly Page Views (sorted descending)
- CNAME Record

**Use this to:**
- Verify the filtered data looks correct
- Check specific subdomains manually while scan runs
- Confirm the range (START_ROW to END_ROW)

In [ ]:
import pandas as pd

# ========================================
# PREVIEW CONFIGURATION (Cell 7 only)
# ========================================
PREVIEW_START_ROW = 1       # First row to preview (1-indexed)
PREVIEW_END_ROW = 100       # Last row to preview

print("=" * 120)
print(f"PREVIEW: Filtered Dataset (Rows {PREVIEW_START_ROW} to {PREVIEW_END_ROW})")
print("=" * 120)
print("")
print(f"Total available after filtering: {len(df_base_filtered):,} rows")
print(f"Sorted by: {SORT_BY} (descending)")
print("")

# Select preview range from base filtered dataset
df_preview = df_base_filtered.iloc[PREVIEW_START_ROW-1:PREVIEW_END_ROW].copy()

# Verify all entries are subdomains (not root domains)
def is_subdomain(domain):
    """Check if domain is a subdomain (has more than 2 parts)"""
    if pd.isna(domain) or not isinstance(domain, str):
        return False
    parts = domain.split('.')
    return len(parts) > 2

# Filter to only show subdomains
subdomain_mask = df_preview['Subdomain'].apply(is_subdomain)
df_preview_subdomains = df_preview[subdomain_mask].copy()
non_subdomain_count = len(df_preview) - len(df_preview_subdomains)

if non_subdomain_count > 0:
    print(f"⚠️  Filtered out {non_subdomain_count} root domains (kept only subdomains)")
    print("")

# Prepare display with row numbers
display_df = df_preview_subdomains.copy()
display_df.insert(0, 'Row', range(PREVIEW_START_ROW, PREVIEW_START_ROW + len(display_df)))

# Select columns to display
display_columns = ['Row', 'Subdomain', 'HTTP_Status', SORT_BY, 'CNAME_Record']
display_columns = [col for col in display_columns if col in display_df.columns]

print("=" * 120)
print(f"Displaying {len(display_df)} subdomains:")
print("=" * 120)
print("")

# Configure pandas display options
pd.set_option('display.max_rows', None)  # Show ALL rows
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.width', 120)

# Display the full preview
print(display_df[display_columns].to_string(index=False))

print("")
print("=" * 120)
print("")
print("📋 Preview Summary:")
print(f"  • Total subdomains shown: {len(display_df)}")
print(f"  • Preview range: Row {PREVIEW_START_ROW} to {PREVIEW_START_ROW + len(display_df) - 1}")
print(f"  • Sorted by: {SORT_BY} (highest first)")
print(f"  • All entries verified as subdomains (not root domains)")
print("")
print("💡 TIP: You can manually review these subdomains while Cell 8 runs the scan")
print("")
print("=" * 120)
print("Next: Configure scan range in Cell 8")
print("=" * 120)

## Cell 8: Deep Scan Execution

**Scan configuration:**
- Mode: `full` (active + passive enumeration)
- Provider: Shopify
- Filters: CNAME contains 'shopify' + HTTP status 3xx/4xx/5xx
- Workers: 2 (optimized for Kaggle)

**Note:** Since inputs are already subdomains, the scanner will:
- Skip subdomain enumeration (saves hours!)
- Go directly to DNS/HTTP validation
- Check for takeover vulnerabilities

**Time:** While this runs, you can review the subdomains from Cell 7

In [ ]:
import subprocess
import sys
import pandas as pd

# ========================================
# SCAN CONFIGURATION (Cell 8 only)
# ========================================
SCAN_START_ROW = 1       # First row to scan (1-indexed)
SCAN_END_ROW = 100       # Last row to scan

print("=" * 80)
print("PREPARING SCAN")
print("=" * 80)
print("")

# Select scan range from base filtered dataset
df_scan = df_base_filtered.iloc[SCAN_START_ROW-1:SCAN_END_ROW].copy()

# Verify all entries are subdomains (not root domains)
def is_subdomain(domain):
    """Check if domain is a subdomain (has more than 2 parts)"""
    if pd.isna(domain) or not isinstance(domain, str):
        return False
    parts = domain.split('.')
    return len(parts) > 2

# Filter to only scan subdomains
subdomain_mask = df_scan['Subdomain'].apply(is_subdomain)
df_scan_subdomains = df_scan[subdomain_mask].copy()
non_subdomain_count = len(df_scan) - len(df_scan_subdomains)

print(f"Scan Range: Rows {SCAN_START_ROW} to {SCAN_END_ROW}")
print(f"Total in range: {len(df_scan)}")
if non_subdomain_count > 0:
    print(f"Filtered out: {non_subdomain_count} root domains")
print(f"Subdomains to scan: {len(df_scan_subdomains)}")
print("")

# Save to file for scanning
targets_file = '/kaggle/working/subdomain-playground/data/all_sources.txt'
df_scan_subdomains['Subdomain'].to_csv(targets_file, index=False, header=False)
print(f"✓ Saved {len(df_scan_subdomains)} targets to: {targets_file}")
print("")

print("=" * 80)
print("STARTING FULL SCAN")
print("=" * 80)
print("")

# Count domains
with open('data/all_sources.txt', 'r') as f:
    domain_count = len(f.readlines())

print(f"Targets: {domain_count} domains")
print("Mode: full (active + passive enumeration)")
print("Workers: 2")
print("")
print("Filters:")
print("  ✓ CNAME contains 'shopify'")
print("  ✓ HTTP status 3xx, 4xx, 5xx")
print("")
print("Features:")
print("  ✓ Automatic subdomain detection (skips enumeration)")
print("  ✓ CNAME blacklist (46 patterns)")
print("  ✓ Cloudflare verification detection")
print("  ✓ Tools: subfinder + findomain + dnsx + httpx + subzy")
print("")
print("Estimated time: 7-9 hours for 10k domains")
print("")
print("=" * 80)
print("")

# Run scan
process = subprocess.Popen(
    [sys.executable, '-u', 'scan.py',
     '-l', 'data/all_sources.txt',
     '--mode', 'full',
     '--require-cname-contains', 'shopify',
     '--filter-status', '3*,4*,5*',
     '--workers', '2'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1
)

# Stream output
for line in process.stdout:
    print(line, end='', flush=True)

process.wait()
print("")
print("=" * 80)
print(f"Scan completed with return code: {process.returncode}")
print("=" * 80)

## Cell 9: View Results

In [ ]:
import json
from pathlib import Path

results_file = Path("/kaggle/working/subdomain-playground/data/scans/shopify_takeover_candidates.json")

if results_file.exists():
    with results_file.open("r") as f:
        results = json.load(f)

    print("=" * 80)
    print("SCAN RESULTS")
    print("=" * 80)
    print("")
    print(f"Total candidates found: {len(results)}")

    # Count by risk level
    risk_counts = {}
    for r in results:
        risk = r.get("risk_level", "unknown")
        risk_counts[risk] = risk_counts.get(risk, 0) + 1

    print("")
    print("Breakdown by risk:")
    for risk, count in sorted(risk_counts.items()):
        print(f"  {risk.upper()}: {count}")

    print("")
    print("=" * 80)
    print("TOP 10 FINDINGS")
    print("=" * 80)

    sorted_results = sorted(results, key=lambda x: x.get("confidence_score", 0), reverse=True)

    for i, r in enumerate(sorted_results[:10], 1):
        print("")
        print(f"{i}. {r['subdomain']}")
        print(f"   CNAME: {r.get('cname', 'N/A')}")
        print(f"   HTTP Status: {r.get('http_status', 'N/A')}")
        print(f"   Risk: {r.get('risk_level', 'N/A')}")
        print(f"   Confidence: {r.get('confidence_score', 0)}")
else:
    print(f"✗ Results file not found: {results_file}")
    print("Make sure the scan completed successfully.")

## Cell 10: Export to CSV

In [ ]:
import json
import pandas as pd
from pathlib import Path

results_file = Path("/kaggle/working/subdomain-playground/data/scans/shopify_takeover_candidates.json")

if not results_file.exists():
    print("✗ Results file not found. Run the scan first.")
else:
    with results_file.open("r") as f:
        results = json.load(f)

    if not results:
        print("✗ No results to export.")
    else:
        df = pd.DataFrame(results)

        # Select key columns
        columns = [
            "subdomain", "cname", "http_status", "risk_level", "confidence_score",
            "cname_chain_count", "final_cname_target", "a_records", "provider"
        ]
        df_export = df[[col for col in columns if col in df.columns]]
        df_export = df_export.sort_values("confidence_score", ascending=False)

        # Save to CSV
        output_csv = Path("/kaggle/working/shopify_takeovers.csv")
        df_export.to_csv(output_csv, index=False)

        print("=" * 80)
        print("CSV EXPORT")
        print("=" * 80)
        print("")
        print(f"✓ Exported {len(df_export)} results to: {output_csv}")
        print("")
        print("Preview (top 20):")
        print(df_export.head(20).to_string(index=False))
        
        # High-risk only
        df_high = df_export[df_export["risk_level"].isin(["critical", "high"])]
        if len(df_high) > 0:
            high_risk_csv = Path("/kaggle/working/shopify_high_risk.csv")
            df_high.to_csv(high_risk_csv, index=False)
            print("")
            print(f"✓ High-risk only ({len(df_high)} results): {high_risk_csv}")
        
        print("")
        print("=" * 80)
        print("✅ COMPLETE!")
        print("=" * 80)